## Groq Llama integration
This is a Groq Llama integration for the [Groq](https://groq.dev) query language. It provides syntax highlighting and code completion for Groq queries as a basis for the rest of the integration.

In [19]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

### Job Description Web Scraping
using webscraping we can streamline our prompts and queries to the specific gob we are applying for to make it easier to use the Groq Llama integration.

In [47]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.linkedin.com/jobs/view/3925135231/?alternateChannel=search&refId=3QOgCeSyOF9SvBfwg9%2Fc2Q%3D%3D&trackingId=VSQ5x%2BaVTaQARN1B87Y3hg%3D%3D")
page_data = loader.load().pop().page_content
print(page_data)





 










MongoDB hiring Software Engineering Internship - Summer 2024/2025 (Sydney) in Sydney, New South Wales, Australia | LinkedIn























 










                Agree & Join LinkedIn
              

      By clicking Continue to join or sign in, you agree to LinkedIn’s User Agreement, Privacy Policy, and Cookie Policy.
    
















      Skip to main content
    



LinkedIn







              Software Engineering Internship - Summer 2024/2025 (Sydney) in Cherrybrook, NSW
 
Expand search






This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. 
 



                        Jobs
                    



                        People
                    



                        Learning
                    












Clear text






Clear text
















 





Clear text









 





Clear text















Cl

In [50]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the first visible job postings and return it in JSON format containing the following keys: `company`, `role`, `experience`, `skills`, `company description` and `role description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "company": "MongoDB",
  "role": "Software Engineering Internship - Summer 2024/2025 (Sydney)",
  "experience": "Internship",
  "skills": [
    "Java",
    "Python",
    "Go",
    "C++",
    "Javascript",
    "Node.js"
  ],
  "company_description": "MongoDB’s mission is to empower innovators to create, transform, and disrupt industries by unleashing the power of software and data. We enable organizations of all sizes to easily build, scale, and run modern applications by helping them modernize legacy workloads, embrace innovation, and unleash AI.",
  "role_description": "We’re seeking curious minds like yours to collaborate with full-time engineers, learning standard development methodologies and gaining practical software engineering experience. Interns are fully integrated software engineers who spend the summer pushing production-level code to make lasting contributions to our products."
}
```


In [52]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'company': 'MongoDB',
 'role': 'Software Engineering Internship - Summer 2024/2025 (Sydney)',
 'experience': 'Internship',
 'skills': ['Java', 'Python', 'Go', 'C++', 'Javascript', 'Node.js'],
 'company_description': 'MongoDB’s mission is to empower innovators to create, transform, and disrupt industries by unleashing the power of software and data. We enable organizations of all sizes to easily build, scale, and run modern applications by helping them modernize legacy workloads, embrace innovation, and unleash AI.',
 'role_description': 'We’re seeking curious minds like yours to collaborate with full-time engineers, learning standard development methodologies and gaining practical software engineering experience. Interns are fully integrated software engineers who spend the summer pushing production-level code to make lasting contributions to our products.'}

Load up your portfolio from a csv file that matches specific techstacks and skills to a project in your portfolio.

In [53]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"Java, Event-Driven Architecture, Microservices...",https://github.com/Biswas57/Tributary-Cluster
1,"Java, Spring Boot, RESTful API Development, Re...",https://github.com/Biswas57/Accenture-SWE-Program
2,"Shell Scripting, POSIX Compliance, Linux, Dash...",https://github.com/Biswas57/GRiD
3,"React, JavaScript, HTML, CSS, Styled-Component...",https://github.com/Biswas57/Recipe-Book-Website
4,"TypeScript, JavaScript, Node.js, Express.js, A...",https://github.com/Biswas57/Toohak


## ChromaDB integration

This vector database will allow this AI tool to filter through our application information to answer application questions and generate emails. This will allow the AI tool to have a better understanding of the user's needs and provide a more accurate response.

Using the vector database we can see below how the database is able to take a simple prompt and use linear algebra to correlate a prompt with the most relevant information in the database. This is done by taking the dot product of the prompt and the database to find the most relevant information.

In [54]:
import chromadb
import uuid

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

job = json_res
job['skills']

['Java', 'Python', 'Go', 'C++', 'Javascript', 'Node.js']

In [55]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/Biswas57/Tributary-Cluster'},
  {'links': 'https://github.com/Biswas57/Accenture-SWE-Program'}],
 [{'links': 'https://github.com/Biswas57/GRiD'},
  {'links': 'https://github.com/Biswas57/Tributary-Cluster'}],
 [{'links': 'https://github.com/Biswas57/Accenture-SWE-Program'},
  {'links': 'https://github.com/Biswas57/GRiD'}],
 [{'links': 'https://github.com/Biswas57/Tributary-Cluster'},
  {'links': 'https://github.com/Biswas57/GRiD'}],
 [{'links': 'https://github.com/Biswas57/Toohak'},
  {'links': 'https://github.com/Biswas57/Recipe-Book-Website'}],
 [{'links': 'https://github.com/Biswas57/Toohak'},
  {'links': 'https://github.com/Biswas57/Recipe-Book-Website'}]]

### Prompt Engineering

The AI will prompt the user for a job description and then scrape the web for the job description and then prompt the user for their portfolio and then scrape the web for the portfolio. The below prompt will be then used to generate a query that will used the above vector database to match the job description to the portfolio and then write the user an email that they can send to the job recruiter.

In [56]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Biswas Simkhada, an aspiring software engineer and a Bachelor of Computer Science student at the University of New South Wales, expected to graduate in December 2026. You have hands-on experience in developing complex software projects, such as creating a Kafka Cluster Emulator using Java and a Kahoot! clone API using TypeScript and JavaScript with Node.js and Express. You have expertise in Java, Python, SQL, JavaScript, and frameworks like Spring Boot, React.js, and Express, as well as experience with CI/CD pipelines, testing frameworks like JUnit and Spock, and various development tools including Git, GitHub, and Vercel.
        Your job is to write a cold email to a job recruiter regarding the job mentioned above, showcasing your expertise in software development, your ability to build scalable and efficient solutions, and your experience with relevant technologies that meet their needs.
        Also, add the most relevant ones from the following links to highlight your portfolio: {link_list}
        Remember you are Biswas Simkhada, a skilled software engineer with a background in AI and software consulting.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineering Internship - Summer 2024/2025 at MongoDB

Dear Hiring Manager,

I am excited to apply for the Software Engineering Internship - Summer 2024/2025 role at MongoDB, as advertised. With a strong passion for software development and a solid foundation in computer science, I am confident that my skills and experience make me an ideal candidate for this position.

As a Bachelor of Computer Science student at the University of New South Wales, I have developed a solid understanding of software engineering principles and have hands-on experience in building complex software projects. My expertise in Java, Python, SQL, JavaScript, and frameworks like Spring Boot, React.js, and Express aligns well with the skills required for this role. Additionally, my experience with CI/CD pipelines, testing frameworks like JUnit and Spock, and various development tools including Git, GitHub, and Vercel will enable me to make a seamless transition into your team.

I